*  a&b


* c

* d

* e

* f

* g

* h

In [121]:
# Import necessary libraries
import pandas as pd
from gurobipy import Model, GRB

# Load datasets
farms = pd.read_csv('/Users/Sam/Downloads/farms.csv')
processing = pd.read_csv('/Users/Sam/Downloads/processing.csv')
centers = pd.read_csv('/Users/Sam/Downloads/centers.csv')

# Initialize the Gurobi model
model = Model('BioAgri Optimization')


In [122]:
# Define dimensions
num_farms = len(farms)
num_plants = len(processing)
num_centers = len(centers)

# Decision variables
# x[i, j]: Tons of raw material transported from farm i to processing plant j
x = model.addVars(num_farms, num_plants, vtype=GRB.CONTINUOUS, name="x")

# y[j, k]: Tons of fertilizer transported from processing plant j to home center k
y = model.addVars(num_plants, num_centers, vtype=GRB.CONTINUOUS, name="y")


In [123]:
# Extract transportation cost data
transport_cost_farm_to_plant = [[farms.iloc[i, 4+j] for j in range(num_plants)] for i in range(num_farms)]
transport_cost_plant_to_center = [[processing.iloc[j, 4+k] for k in range(num_centers)] for j in range(num_plants)]

# Objective function: Minimize total cost
model.setObjective(
    sum(x[i, j] * (farms.loc[i, 'Cost_Per_Ton'] + transport_cost_farm_to_plant[i][j])
        for i in range(num_farms) for j in range(num_plants)) +
    sum(y[j, k] * (processing.loc[j, 'Processing_Cost_Per_Ton'] + transport_cost_plant_to_center[j][k])
        for j in range(num_plants) for k in range(num_centers)),
    GRB.MINIMIZE
)


In [124]:
# 1. Farm supply constraints
for i in range(num_farms):
    model.addConstr(sum(x[i, j] for j in range(num_plants)) <= farms.loc[i, 'Bio_Material_Capacity_Tons'], 
                    name=f"FarmSupply_{i}")

# 2. Plant capacity constraints
for j in range(num_plants):
    model.addConstr(sum(x[i, j] for i in range(num_farms)) <= processing.loc[j, 'Capacity_Tons'], 
                    name=f"PlantCapacity_{j}")

# 3. Home center demand constraints
for k in range(num_centers):
    model.addConstr(sum(y[j, k] for j in range(num_plants)) == centers.loc[k, 'Requested_Demand_Tons'], 
                    name=f"CenterDemand_{k}")

# 4. Flow balance constraints
for j in range(num_plants):
    model.addConstr(sum(x[i, j] for i in range(num_farms)) == sum(y[j, k] for k in range(num_centers)), 
                    name=f"FlowBalance_{j}")


In [125]:
# Optimize the model
model.optimize()

# Output results
if model.status == GRB.OPTIMAL:
    print("Optimal Cost:", model.objVal)
    print("Decision Variables:")
    for v in model.getVars():
        if v.x > 0:  # Only print variables with non-zero values
            print(f"{v.varName}: {v.x}")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.2.0 24C101)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 387 rows, 6318 columns and 17118 nonzeros
Model fingerprint: 0x3aa78cc0
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+00, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 3e+04]
Presolve time: 0.00s
Presolved: 387 rows, 6318 columns, 17118 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9457439e+05   2.902000e+04   0.000000e+00      0s
     330    2.2970900e+06   0.000000e+00   0.000000e+00      0s

Solved in 330 iterations and 0.01 seconds (0.02 work units)
Optimal objective  2.297089973e+06
Optimal Cost: 2297089.972722625
Decision Variables:
x[3,8]: 367.0
x[4,5]: 499.0
x[5,15]: 417.0
x[13,5]: 526.0
x[14,8]: 441.0
x[25,1]: 564.0
x[30,8]: 468.0
x[34,0]: 476.0
x[35,8]: 519.0
x[37,5]: 379.0
x[3

* c

In [130]:
# Constraints for the optimization model

# Extract transportation cost data
transport_cost_farm_to_plant = [[farms.iloc[i, 4+j] for j in range(num_plants)] for i in range(num_farms)]
transport_cost_plant_to_center = [[processing.iloc[j, 4+k] for k in range(num_centers)] for j in range(num_plants)]

# Objective function: Minimize total cost
model.setObjective(
    sum(x[i, j] * (farms.loc[i, 'Cost_Per_Ton'] + transport_cost_farm_to_plant[i][j])
        for i in range(num_farms) for j in range(num_plants)) +
    sum(y[j, k] * (processing.loc[j, 'Processing_Cost_Per_Ton'] + transport_cost_plant_to_center[j][k])
        for j in range(num_plants) for k in range(num_centers)),
    GRB.MINIMIZE
)

# 1. Farm supply constraints
for i in range(num_farms):
    model.addConstr(sum(x[i, j] for j in range(num_plants)) <= farms.loc[i, 'Bio_Material_Capacity_Tons'], 
                    name=f"FarmSupply_{i}")

# 2. Plant capacity constraints
for j in range(num_plants):
    model.addConstr(sum(x[i, j] for i in range(num_farms)) <= processing.loc[j, 'Capacity_Tons'], 
                    name=f"PlantCapacity_{j}")

# 3. Home center demand constraints
for k in range(num_centers):
    model.addConstr(sum(y[j, k] for j in range(num_plants)) == centers.loc[k, 'Requested_Demand_Tons'], 
                    name=f"CenterDemand_{k}")

# 4. Flow balance constraints
for j in range(num_plants):
    model.addConstr(sum(x[i, j] for i in range(num_farms)) == sum(y[j, k] for k in range(num_centers)), 
                    name=f"FlowBalance_{j}")

# 5. Regional restriction constraints
# Processing plants can only send fertilizer to home centers in the same region
for j in range(num_plants):
    plant_region = processing.loc[j, 'Region']
    for k in range(num_centers):
        center_region = centers.loc[k, 'Region']
        if plant_region != center_region:
            model.addConstr(y[j, k] == 0, name=f"RegionalRestriction_Plant{j}_Center{k}")

# Optimize the model
model.optimize()

# Output results
if model.status == GRB.OPTIMAL:
    print("Optimal Cost (Regional Restriction):", model.objVal)
    print("Decision Variables:")
    for v in model.getVars():
        if v.x > 0:  # Only print variables with non-zero values
            print(f"{v.varName}: {v.x}")
else:
    print("No optimal solution found under regional restrictions.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.2.0 24C101)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7428 rows, 6318 columns and 74352 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+00, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 3e+04]
LP warm-start: use basis

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.7381795e+06   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  5.738179524e+06
Optimal Cost (Regional Restriction): 5738179.523795238
Decision Variables:
x[2,9]: 516.0
x[10,15]: 256.0
x[11,17]: 261.0
x[15,1]: 289.0
x[17,9]: 525.0
x[18,9]: 229.0
x[19,5]: 162.0
x[19,12]: 396.0
x[20,8]: 487.0
x[23,5]: 434.0
x[28,17]: 526.0
x[31,8]: 108.0
x[32,8]: 330.0
x[39,9]: 576.0
x[44,12]: 461.0
x[48,9]: 282.0
x[52,12]: 391.0
x[57,16]:

* d

In [131]:
#Constraints for the optimization model

# 1. Farm supply constraints
for i in range(num_farms):
    model.addConstr(sum(x[i, j] for j in range(num_plants)) <= farms.loc[i, 'Bio_Material_Capacity_Tons'], 
                    name=f"FarmSupply_{i}")

# 2. Plant capacity constraints
for j in range(num_plants):
    model.addConstr(sum(x[i, j] for i in range(num_farms)) <= processing.loc[j, 'Capacity_Tons'], 
                    name=f"PlantCapacity_{j}")

# 3. Home center demand constraints
for k in range(num_centers):
    model.addConstr(sum(y[j, k] for j in range(num_plants)) == centers.loc[k, 'Requested_Demand_Tons'], 
                    name=f"CenterDemand_{k}")

# 4. Flow balance constraints
for j in range(num_plants):
    model.addConstr(sum(x[i, j] for i in range(num_farms)) == sum(y[j, k] for k in range(num_centers)), 
                    name=f"FlowBalance_{j}")

# 5. Quality restriction constraints
# Only raw materials with quality levels 3 and 4 are allowed
for i in range(num_farms):
    if farms.loc[i, 'Quality'] < 3:  # If quality is not 3 or 4
        for j in range(num_plants):
            model.addConstr(x[i, j] == 0, name=f"QualityRestriction_Farm{i}_Plant{j}")

# Optimize the model
model.optimize()

# Output results
if model.status == GRB.OPTIMAL:
    print("Optimal Cost (Highest Quality Only):", model.objVal)
    print("Decision Variables:")
    for v in model.getVars():
        if v.x > 0:  # Only print variables with non-zero values
            print(f"{v.varName}: {v.x}")
else:
    print("No optimal solution found with quality restrictions.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.2.0 24C101)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10947 rows, 6318 columns and 94602 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+00, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 3e+04]
LP warm-start: use basis

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.7381795e+06   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  5.738179524e+06
Optimal Cost (Highest Quality Only): 5738179.523795238
Decision Variables:
x[2,9]: 516.0
x[10,15]: 256.0
x[11,17]: 261.0
x[15,1]: 289.0
x[17,9]: 525.0
x[18,9]: 229.0
x[19,5]: 162.0
x[19,12]: 396.0
x[20,8]: 487.0
x[23,5]: 434.0
x[28,17]: 526.0
x[31,8]: 108.0
x[32,8]: 330.0
x[39,9]: 576.0
x[44,12]: 461.0
x[48,9]: 282.0
x[52,12]: 391.0
x[57,16]

* e

In [128]:
# # Constraints for the optimization model - Part 1

# # Extract transportation cost data
# transport_cost_farm_to_plant = [[farms.iloc[i, 4+j] for j in range(num_plants)] for i in range(num_farms)]
# transport_cost_plant_to_center = [[processing.iloc[j, 4+k] for k in range(num_centers)] for j in range(num_plants)]

# # Objective function: Minimize total cost
# model.setObjective(
#     sum(x[i, j] * (farms.loc[i, 'Cost_Per_Ton'] + transport_cost_farm_to_plant[i][j])
#         for i in range(num_farms) for j in range(num_plants)) +
#     sum(y[j, k] * (processing.loc[j, 'Processing_Cost_Per_Ton'] + transport_cost_plant_to_center[j][k])
#         for j in range(num_plants) for k in range(num_centers)),
#     GRB.MINIMIZE
# )

# # 1. Farm supply constraints
# for i in range(num_farms):
#     model.addConstr(sum(x[i, j] for j in range(num_plants)) <= farms.loc[i, 'Bio_Material_Capacity_Tons'], 
#                     name=f"FarmSupply_{i}")

# # 2. Plant capacity constraints
# for j in range(num_plants):
#     model.addConstr(sum(x[i, j] for i in range(num_farms)) <= processing.loc[j, 'Capacity_Tons'], 
#                     name=f"PlantCapacity_{j}")

# # 3. Home center demand constraints
# for k in range(num_centers):
#     model.addConstr(sum(y[j, k] for j in range(num_plants)) == centers.loc[k, 'Requested_Demand_Tons'], 
#                     name=f"CenterDemand_{k}")

# # 4. Flow balance constraints
# for j in range(num_plants):
#     model.addConstr(sum(x[i, j] for i in range(num_farms)) == sum(y[j, k] for k in range(num_centers)), 
#                     name=f"FlowBalance_{j}")

# # 5. Sourcing risk mitigation constraint
# # Each plant can process no more than 3% of the total raw material sourced
# total_supply = farms['Bio_Material_Capacity_Tons'].sum()
# for j in range(num_plants):
#     model.addConstr(sum(x[i, j] for i in range(num_farms)) <= 0.03 * total_supply, 
#                     name=f"SourcingRiskMitigation_Plant{j}")

# # Optimize the model
# model.optimize()

# # Output results
# if model.status == GRB.OPTIMAL:
#     print("Optimal Cost (Sourcing Risk Mitigation):", model.objVal)
#     print("Decision Variables:")
#     for v in model.getVars():
#         if v.x > 0:  # Only print variables with non-zero values
#             print(f"{v.varName}: {v.x}")
# else:
#     print("No optimal solution found with sourcing risk mitigation.")


In [129]:
# # Constraints for the optimization model - Part 2

# # Extract transportation cost data
# transport_cost_farm_to_plant = [[farms.iloc[i, 4+j] for j in range(num_plants)] for i in range(num_farms)]
# transport_cost_plant_to_center = [[processing.iloc[j, 4+k] for k in range(num_centers)] for j in range(num_plants)]

# # Objective function: Minimize total cost
# model.setObjective(
#     sum(x[i, j] * (farms.loc[i, 'Cost_Per_Ton'] + transport_cost_farm_to_plant[i][j])
#         for i in range(num_farms) for j in range(num_plants)) +
#     sum(y[j, k] * (processing.loc[j, 'Processing_Cost_Per_Ton'] + transport_cost_plant_to_center[j][k])
#         for j in range(num_plants) for k in range(num_centers)),
#     GRB.MINIMIZE
# )


# # 1. Farm supply constraints
# for i in range(num_farms):
#     model.addConstr(sum(x[i, j] for j in range(num_plants)) <= farms.loc[i, 'Bio_Material_Capacity_Tons'], 
#                     name=f"FarmSupply_{i}")

# # 2. Plant capacity constraints
# for j in range(num_plants):
#     model.addConstr(sum(x[i, j] for i in range(num_farms)) <= processing.loc[j, 'Capacity_Tons'], 
#                     name=f"PlantCapacity_{j}")

# # 3. Home center demand constraints
# for k in range(num_centers):
#     model.addConstr(sum(y[j, k] for j in range(num_plants)) == centers.loc[k, 'Requested_Demand_Tons'], 
#                     name=f"CenterDemand_{k}")

# # 4. Flow balance constraints
# for j in range(num_plants):
#     model.addConstr(sum(x[i, j] for i in range(num_farms)) == sum(y[j, k] for k in range(num_centers)), 
#                     name=f"FlowBalance_{j}")

# # 5. Supply risk mitigation constraint
# # A plant can supply no more than 50% of the total demand of a single home center
# for j in range(num_plants):
#     for k in range(num_centers):
#         model.addConstr(y[j, k] <= 0.5 * centers.loc[k, 'Requested_Demand_Tons'], 
#                         name=f"SupplyRiskMitigation_Plant{j}_Center{k}")

# # Optimize the model
# model.optimize()

# # Output results
# if model.status == GRB.OPTIMAL:
#     print("Optimal Cost (Supply Risk Mitigation):", model.objVal)
#     print("Decision Variables:")
#     for v in model.getVars():
#         if v.x > 0:  # Only print variables with non-zero values
#             print(f"{v.varName}: {v.x}")
# else:
#     print("No optimal solution found with supply risk mitigation.")
